In [1]:
import joblib
import pickle
import re
import spacy

In [2]:
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install joblib

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB 93.5 kB/s eta 0:02:17
     ---------------------------------------- 0.0/12.8 MB 93.5 kB/s eta 0:02:17
     ---------------------------------------- 0.0/12.8 MB 93.5 kB/s eta 0:02:17
     ---------------------------------------- 0.0/12.8 MB 93.5 kB/s eta 0:02:17
     ---------------------------

ERROR: Exception:
Traceback (most recent call last):
  File "D:\PROJECT_NEW\Codesphere_Labs\TODO\venv\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "D:\PROJECT_NEW\Codesphere_Labs\TODO\venv\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "D:\PROJECT_NEW\Codesphere_Labs\TODO\venv\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "D:\PROJECT_NEW\Codesphere_Labs\TODO\venv\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
                  ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\rasintha_j\AppData\Local\Programs\Python\Python311\Lib\http\client.py", line 465, in read
    s = self.fp.read(amt)
        ^^^^^^^^^^^^^^^^^
  File "C:\Users\ra

In [3]:
class ExtractFromToDo:
    def __init__(self):
        # Load the spaCy model
        self.nlp = spacy.load("en_core_web_sm")

    def extract_locations(self, text):
        # Process the sentence using spaCy
        doc = self.nlp(text)
        # Extract entities
        locations = [ent.text for ent in doc.ents if ent.label_ in ["GPE", "LOC", "FAC"]]
        return locations

    def extract_time(self, text):
        time_pattern = re.compile(r'\b(?:[0-1]?[0-9]|2[0-3])[:.][0-5][0-9](?:\s*(?:AM|PM|am|pm|Pm|Am))?\b')
        matches = re.findall(time_pattern, text)
        return matches

    def extract_dates(self, text):
        date_pattern = re.compile(r'\b(?:\d{1,2}[-./]\d{1,2}[-./]\d{2,4}|\d{4}[-./]\d{1,2}[-./]\d{1,2})\b')
        matches = re.findall(date_pattern, text)
        return matches

    def extract_title(self, text):
        doc = self.nlp(text)
        # Extracting determiners (DET) and nouns (NOUN)
        det_noun_pairs = [(token.text, token.head.text) for token in doc if token.pos_ == "DET" and token.head.pos_ == "NOUN"]
        # Return the extracted pairs as a list of titles
        titles = [f"{det} {noun}" for det, noun in det_noun_pairs]
        return titles

    def extract_info_from_sentence(self, sentence):
        locations = self.extract_locations(sentence)
        times = self.extract_time(sentence)
        dates = self.extract_dates(sentence)
        titles = self.extract_title(sentence)

        max_length = max(len(locations), len(times), len(dates), len(titles))

        event_info_list = []
        for i in range(max_length):
            event_info = {
                "Title": titles[i] if i < len(titles) else "Empty",
                "Venue": locations[i] if i < len(locations) else "Empty",
                "Date": dates[i] if i < len(dates) else "Empty",
                "Time": times[i] if i < len(times) else "Empty"
            }
            event_info_list.append(event_info)

        return event_info_list

    def determine_missing_info(self, event_info_list):
        missing_info_list = []

        # Check if all fields are present in any entry
        all_present = all(info != "Empty" for info in event_info_list[0].values())

        if not all_present:
            # At least one field is missing, determine missing info
            for event_info in event_info_list:
                missing_info = {}
                for key, value in event_info.items():
                    if value == "Empty":
                        # Determine the missing information based on available data
                        if key == "Title":
                            missing_info[key] = "Event Name"  # Placeholder for missing event name
                        elif key == "Venue":
                            missing_info[key] = "Venue Name"  # Placeholder for missing venue name
                        elif key == "Date":
                            missing_info[key] = "Date"  # Placeholder for missing date
                        elif key == "Time":
                            missing_info[key] = "Time"  # Placeholder for missing time
                    else:
                        missing_info[key] = value
                missing_info_list.append(missing_info)

        return missing_info_list

# Example usage:
extractor = ExtractFromToDo()

# Example sentences provided from outside as arguments
input_sentence = input()
# Extract information from the sentence
event_info_list = extractor.extract_info_from_sentence(input_sentence)

# Print the extracted information
print("Extracted Information:")
for event_info in event_info_list:
    print("Event :", event_info["Title"], "Venue :", event_info["Venue"], "Date :", event_info["Date"], "Time :", event_info["Time"])

# Determine missing information
missing_info_list = extractor.determine_missing_info(event_info_list)

# Print the missing information
if missing_info_list:
    print("\nMissing Information:")
    for missing_info in missing_info_list:
        print("Event :", missing_info["Title"], "Venue :", missing_info["Venue"], "Date :", missing_info["Date"], "Time :", missing_info["Time"])

 I have a wedding in Waththala at 8.00pm in 2023/02/12


Extracted Information:
Event : a wedding Venue : Waththala Date : 2023/02/12 Time : 8.00pm


In [6]:
from joblib import dump

In [10]:
joblib.dump(extractor, './../saveModels/model.joblib')

['./../saveModels/model.joblib']